# We save the categorical from last lab in one variable

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder

In [3]:
data = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')

In [4]:
columns2 = []
for col in data.columns:
    columns2.append(re.sub(" ","_", col).lower())
data.columns = columns2

In [5]:
categorical_df = data.select_dtypes(object)

In [6]:
categorical_df = categorical_df.drop(columns = ['customer','effective_to_date'])

In [7]:
mapping = {'No':0,'Yes':1}
categorical_df = categorical_df.replace({'response': mapping})
mapping = {'M':0,'F':1}
categorical_df = categorical_df.replace({'gender': mapping})

In [8]:
mapping = {'Offer1':1,'Offer2':2,'Offer3':3,'Offer4':4}
categorical_df = categorical_df.replace({'renew_offer_type': mapping})

In [21]:
categorical_df['policy_type'] = categorical_df.policy_type.apply(lambda x: x.split(" ")[0])
categorical_df['policy'] = categorical_df.policy.apply(lambda x: x[-1])

In [29]:
categorical_df['policy'] = pd.to_numeric(categorical_df['policy'])

In [30]:
mapping = {'Basic':0,'Extended':1,'Premium':2}
categorical_df = categorical_df.replace({'coverage': mapping})
mapping = {'Small':0,'Medsize':1,'Large':2}
categorical_df = categorical_df.replace({'vehicle_size': mapping})

In [31]:
# Split numerical and rest
categorical_df_num = categorical_df.drop(columns =['state','education','employmentstatus','location_code','policy_type','sales_channel','vehicle_class','marital_status'])
categorical_df_rest = categorical_df[['state','education','employmentstatus','location_code','policy_type','sales_channel','vehicle_class','marital_status']]

In [32]:
def Luxury_car(x):
    if ('Luxury' in x) or ('Sports' in x):
        x = "Luxury"
    return x

def High_education(x):
    if ('Master' in x) or ('Doctor' in x):
        x = "High Education"
    return x
    
def Employment(x):
    if ('Medical' in x) or ('Disabled' in x) or ('Retired' in x):
        x = "other"
    return x

categorical_df_rest2 = categorical_df_rest.copy()
categorical_df_rest2['vehicle_class'] = categorical_df_rest2['vehicle_class'].apply(Luxury_car)
categorical_df_rest2['education'] = categorical_df_rest2['education'].apply(High_education)
categorical_df_rest2['employmentstatus'] = categorical_df_rest2['employmentstatus'].apply(Employment)

In [33]:
categorical_df_rest2.to_pickle("categorical_df_rest.pkl")
categorical_df_num.to_pickle("categorical_df_num.pkl")

In [34]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(categorical_df_rest2)
onehotlabels = enc.transform(categorical_df_rest2).toarray()
categorical_df_enc = pd.DataFrame(onehotlabels,columns = np.concatenate(enc.categories_))

In [35]:
categorical_total = pd.concat([categorical_df_num,categorical_df_enc], axis = 1)

In [36]:
categorical_total.to_pickle("categorical_total.pkl")